In [13]:
import pandas as pd
data = pd.read_csv('Processed.csv')
data.head()

,Unnamed: 0,loan_amnt,term,int_rate,installment,annual_inc,loan_status,dti,earliest_cr_line,open_acc,...,home_ownership_RENT,zip_code_05113,zip_code_11650,zip_code_22690,zip_code_29597,zip_code_30723,zip_code_48052,zip_code_70466,zip_code_86630,zip_code_93700
0,0,10000.0,36,11.44,329.48,117000.0,0,26.24,1990,16.0,...,1,0,0,1,0,0,0,0,0,0
1,1,8000.0,36,11.99,265.68,65000.0,0,22.05,2004,17.0,...,0,1,0,0,0,0,0,0,0,0
2,2,15600.0,36,10.49,506.97,43057.0,0,12.79,2007,13.0,...,1,1,0,0,0,0,0,0,0,0
3,3,7200.0,36,6.49,220.65,54000.0,0,2.60,2006,6.0,...,1,0,0,0,0,0,0,0,0,0
4,4,24375.0,60,17.27,609.33,55000.0,1,33.95,1999,13.0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

defaulter = data[data.loan_status == 1]
num_defaulter = len(defaulter)
paid = data[data.loan_status == 0]
num_paid = len(paid)

defaulter_train, defaulter_test = train_test_split (defaulter, test_size=0.3)
paid_train, paid_test = train_test_split (paid, test_size=0.3)

In [ ]:
test = pd.concat ([defaulter_test, paid_test])
X_defaulter = defaulter.drop('loan_status', axis=1)
y_defaulter = defaulter.loan_status

X_paid = paid.drop('loan_status', axis=1)
y_paid = paid.loan_status

In [15]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(defaulter_train)

c:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_continuous_distns.py:5320: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
c:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
c:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_continuous_distns.py:639: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
c:\Users\gupta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\_continuous_distns.py:615: Ru

In [16]:
sample = model.sample (len(paid_train) - len(defaulter_train))
sample.head()

,Unnamed: 0,loan_amnt,term,int_rate,installment,annual_inc,loan_status,dti,earliest_cr_line,open_acc,...,home_ownership_RENT,zip_code_05113,zip_code_11650,zip_code_22690,zip_code_29597,zip_code_30723,zip_code_48052,zip_code_70466,zip_code_86630,zip_code_93700
0,224585,21172.0,53,11.46,530.48,64852.28,1,15.17,2003,13.0,...,0,0,0,0,0,0,0,0,0,0
1,357588,4558.0,36,15.49,177.73,31089.05,1,28.38,1997,7.0,...,1,0,0,0,0,0,0,0,1,0
2,184133,29620.0,60,23.14,694.67,139719.72,1,11.58,2003,8.0,...,1,0,0,1,0,0,0,0,0,0
3,167317,9102.0,48,14.01,247.78,31663.26,1,16.61,1996,15.0,...,0,0,0,0,0,0,0,0,0,0
4,65853,6222.0,48,13.32,183.64,34160.64,1,33.41,1989,4.0,...,1,0,0,0,0,0,0,0,1,0


In [17]:
train = pd.concat([paid_train, defaulter_train, sample])
test = pd.concat ([paid_test, defaulter_test])

X_train = train.drop ('loan_status', axis=1)
y_train = train.loan_status

X_test = test.drop ('loan_status', axis=1)
y_test= test.loan_status

In [19]:
w_p = train.loan_status.value_counts()[0] / train.shape[0]
w_n = train.loan_status.value_counts()[1] / train.shape[0]

print(f"Weight of positive values {w_p}")
print(f"Weight of negative values {w_n}")

Weight of positive values 0.5
Weight of negative values 0.5


In [20]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc

def print_score(true, pred, train=True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

In [22]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier()

xgb_clf.fit(X_train, y_train)

y_train_pred = xgb_clf.predict(X_train)
y_test_pred = xgb_clf.predict(X_test)

print_score(y_test, y_test_pred, train=False)

Test Result:
Accuracy Score: 88.45%
_______________________________________________
CLASSIFICATION REPORT:
                      0             1  accuracy      macro avg   weighted avg
precision      0.885352      0.876942  0.884453       0.881147       0.883702
recall         0.983632      0.478007  0.884453       0.730819       0.884453
f1-score       0.931908      0.618745  0.884453       0.775327       0.870480
support    95309.000000  23257.000000  0.884453  118566.000000  118566.000000
_______________________________________________
Confusion Matrix: 
 [[93749  1560]
 [12140 11117]]

